# 聚类的评估指标

聚类的评估不像分类那么容易,我们训练的时候只知道特征没有标签,那自然评估的时候和标签会对不上.下面是几种常见的聚类性能评估方式

## 有标签参考的情况下

有时候我们会拿带标签数据去掉标签做聚类,这种时候以下的评估指标就可以有用了

### 调整兰德系数(Adjusted Rand  Index)


调整兰德系数,用于体现不考虑标签顺序的相似性.


$$ \text{RI} = \frac{a + b}{C_2^{n_{samples}}} $$


其中 $C_2^{n_{samples}}$ 是数据集中可能的数据对(pairs)的总数(不排序).

然而,RI评分不能保证随机标签任务(random label assignments)将获得接近零的值(特别是如果簇的数量与采样数量相同的数量级).


为了抵消这种影响,我们可以通过定义`adjusted Rand index`(调整后的Rand index)来折现(discount)随机标签的预期$RI E[\text{RI}]$ ,如下所示:

$$ \text{ARI} = \frac{\text{RI} - E[\text{RI}]}{\max(\text{RI}) - E[\text{RI}]}$$

`adjusted rand score`是对称的(symmetric)-- 交换参数不会改变得分.它可以作为共识度量(consensus measure),

其取值范围在[-1,1],而负数或者得分接近0说明效果不佳;越大说明效果越好.

它的缺点是必须要有真实标签,这在现实中怕是比较难做到.

### 互信息(Mutual Information)

互信息常常作为离散数据关联程度的度量,针对聚类效果评估,常见的有:

+ Normalized Mutual Information(NMI) 
+ Adjusted Mutual Information(AMI)


假设两个标签分配(相同的N个对象), 和 V.它们的熵是一个分区集合的不确定性量,定义如下:

$$ H(U) = - \sum_{i=1}^{|U|}P(i)\log(P(i)) $$

其中 $P(i) = \frac{|U_i|}{N}$ 是从U中随机选取的对象到类$ U_i$的概率.

同样对于 V:

$$ H(V) = - \sum_{j=1}^{|V|}P'(j)\log(P'(j)) $$

使用 $P'(j) = |V_j| / N$.U和V之间的mutual information(MI)由下式计算:

$$ \text{MI}(U, V) = \sum_{i=1}^{|U|}\sum_{j=1}^{|V|}P(i, j)\log\left(\frac{P(i,j)}{P(i)P'(j)}\right) $$

其中$ P(i, j) = |U_i \cap V_j| / N $是随机选择的对象落入两个类的概率 $U_i$ 和$ V_j $。

也可以用设定的基数表达式表示:

$$\text{MI}(U, V) = \sum_{i=1}^{|U|} \sum_{j=1}^{|V|} \frac{|U_i \cap V_j|}{N}\log\left(\frac{N|U_i \cap V_j|}{|U_i||V_j|}\right)$$

normalized mutual information 被定义为

$$ \text{NMI}(U, V) = \frac{\text{MI}(U, V)}{\sqrt{H(U)H(V)}}$$

mutual information 的值以及 normalized variant(标准化变量)的值不会因 chance而被调整,随着不同标签簇的数量的增加,不管标签分配之间的"mutual information"的实际数量如何,都会趋向于增加.

mutual information 的期望值可以用 Vinh,Epps 和 Bailey,(2009) 的以下公式来计算.在这个方程式中,$ a_i = |U_i| $($U_i$中元素的数量) 和 $b_j = |V_j| $($V_j$ 中元素的数量).

$$ E[\text{MI}(U,V)]=\sum_{i=1}^|U| \sum_{j=1}^|V| \sum_{n_{ij}=(a_i+b_j-N)^+
}^{\min(a_i, b_j)} \frac{n_{ij}}{N}\log \left( \frac{ N.n_{ij}}{a_i b_j}\right)
\frac{a_i!b_j!(N-a_i)!(N-b_j)!}{N!n_{ij}!(a_i-n_{ij})!(b_j-n_{ij})!
(N-a_i-b_j+n_{ij})!} $$

使用期望值, 然后可以使用与 adjusted Rand index 相似的形式来计算调整后的 mutual information:

$$ \text{AMI} = \frac{\text{MI} - E[\text{MI}]}{\max(H(U), H(V)) - E[\text{MI}]}$$

互信息的取值范围是[0,1],它的缺点和上面一样,得有个真实标签才可以

### 同质性(homogeneity),完整性(completeness)和 V-measure

有点类似分类问题的精确率,召回率和f1.


同质性和completeness的得分由下面公式给出:

$$h = 1 - \frac{H(C|K)}{H(C)}$$

$$c = 1 - \frac{H(K|C)}{H(K)}$$

其中$ H(C|K) $是 给定簇分配的类的条件熵,由下式给出:

$$ H(C|K) = - \sum_{c=1}^{|C|} \sum_{k=1}^{|K|} \frac{n_{c,k}}{n}
\cdot \log\left(\frac{n_{c,k}}{n_k}\right) $$

并且H(C)是类的熵,并且由下式给出:

$$H(C) = - \sum_{c=1}^{|C|} \frac{n_c}{n} \cdot \log\left(\frac{n_c}{n}\right)$$

n个样本总数,$ n_c$ 和$ n_k $分别属于 c 类和簇 k 的样本数,最后$ n_{c,k}$ 分配给簇 k 的类 c 的样本数.

给定类的条件熵$ H(K|C)$ 和 簇的熵$ H(K)$ 以对称方式定义.

Rosenberg 和 Hirschberg 进一步定义 V-measure 作为同质性和完整性的调和平均数:

$$ v = 2 \cdot \frac{h \cdot c}{h + c} $$

这几个参数与上面一样取之范围为[0,1],越大说明越好.

### Fowlkes-Mallows 分数

Fowlkes-Mallows 分数被定义为 成对的准确率和 recall召回率的几何平均值:

$$ \text{FMI} = \frac{\text{TP}}{\sqrt{(\text{TP} + \text{FP}) (\text{TP} + \text{FN})}} $$

取值范围为[0,1].较高的值代表效果越好.


## 无标签参考情况下的度量

更多的情况下,聚类就是为了处理无标签数据的,这种时候压根就没有标签,聚类效果的度量就只能从数据自己身上找了.

### Silhouette 系数

该系数基于两个集合间的平均距离.

$$ s = \frac{b - a}{max(a, b)} $$


+ a: 样本与同一类别中所有其他点之间的平均距离。
+ b: 样本与 下一个距离最近的簇 中的所有其他点之间的平均距离。

Silhouette系数取值为[-1,1],越高说明聚类效果越好

silhouette系数的缺点是凸簇通常比其他类型的簇更高.

### Calinski-Harabaz 指数

对于k簇,Calinski-Harabaz得分s是作为簇间色散平均值与群内色散之间的比值给出的:

$$ s(k) = \frac{\mathrm{Tr}(B_k)}{\mathrm{Tr}(W_k)} \times \frac{N - k}{k - 1} $$

其中$B_K$是 组间色散矩阵,$W_K$是由以下定义的群内色散矩阵:

$$ W_k = \sum_{q=1}^k \sum_{x \in C_q} (x - c_q) (x - c_q)^T$$

$$ B_k = \sum_q n_q (c_q - c) (c_q - c)^T $$

N为数据中的点数,$C_q$为簇q中的点集$c_q$为簇q的中心,c 为 E 的中心,$n_q$为簇q中的点数.

该指标的取值范围为

这个指数的优点是:

+ 当簇密集且分离较好时,分数更高
+ 得分计算很快

缺点和上面一样:

+ 凸的簇的该指数通常高于其他类型的簇.

## ***使用sklearn做模型评估***

sklearn提供了一些接口来做聚类模型的评估

接口|说明
---|---
`metrics.adjusted_mutual_info_score(…)`|调整互信息指数
`metrics.adjusted_rand_score(labels_true, …)`|调整兰德系数
`metrics.calinski_harabaz_score(X, labels)`|计算calinski和harabaz评分
`metrics.completeness_score(labels_true, …)`|给定一个基本事实的集群标注的完备度量
`metrics.fowlkes_mallows_score(labels_true, …)`|Fowlkes-Mallows分数
`metrics.homogeneity_completeness_v_measure(…)`|计算同质性(homogeneity),完整性(completeness)和 V-measure
`metrics.homogeneity_score(labels_true, …)`|计算同质性(homogeneity)
`metrics.mutual_info_score(labels_true, …)`|互信息指数
`metrics.normalized_mutual_info_score(…)`|标准化互信息指数
`metrics.silhouette_score(X, labels[, …])`|计算Silhouette系数
`metrics.silhouette_samples(X, labels[, metric])`|为每个样本计算Silhouette系数
`metrics.v_measure_score(labels_true, labels_pred)`|计算V-measure